In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import sqlite3

# Disable scientific notation in pandas
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# Load Dataset
df = pd.read_csv(r"C:\Users\fiona\Downloads\archive\Amazon Sale Report.csv",low_memory=False)

print(df.head())
print(df.info())
print(df.isna().sum())

# Drop Columns
df.drop(columns=["fulfilled-by", "Unnamed: 22", "promotion-ids"], inplace=True)

# Fill missing values
df["Courier Status"] = df["Courier Status"].fillna("Unknown")
df["currency"] = df["currency"].fillna("Unknown")
df["Amount"] = df["Amount"].fillna(df["Amount"].mean())
for col in ["ship-city", "ship-state", "ship-postal-code", "ship-country"]:
    df[col] = df[col].fillna(df[col].mode()[0])
print(df.isna().sum())

# Data type conversion
df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y", errors="coerce")
df["ship-postal-code"] = df["ship-postal-code"].astype(str)
print(df.info())

# SQLite connection
conn = sqlite3.connect(":memory:")

# Push the DataFrame into SQL
df.to_sql("sales", conn, index=False, if_exists="replace")

# Top 10 Best-Selling Products by Quantity
query1 = """
SELECT SKU, SUM(Qty) as Total_Quantity
FROM sales
GROUP BY SKU
ORDER BY Total_Quantity DESC
LIMIT 10;
"""
top_products = pd.read_sql_query(query1, conn)
print("\nTop 10 Best-Selling Products:\n", top_products.to_string(index=False))

# Monthly Revenue Trend
query2 = """
SELECT strftime('%Y-%m', Date) as Month, SUM(Amount) as Revenue
FROM sales
WHERE Date IS NOT NULL
GROUP BY Month
ORDER BY Month;
"""
monthly_revenue = pd.read_sql_query(query2, conn)
print("\nMonthly Revenue Trend:\n", monthly_revenue.to_string(index=False))

# Top Customers by Spend
query3 = """
SELECT `Order ID`, SUM(Amount) as Total_Spend
FROM sales
GROUP BY `Order ID`
ORDER BY Total_Spend DESC
LIMIT 10;
"""
top_customers = pd.read_sql_query(query3, conn)
print("\nTop Customers by Spend:\n", top_customers.to_string(index=False))

# Key performance indicators
# Total Revenue
print("\n---------Key Performance Indicators (KPIs):-----------")
total_revenue = df["Amount"].sum()
print(f"Total Revenue: {total_revenue:,.2f}")

# Average Order Value
avg_order_value = df.groupby("Order ID")["Amount"].sum().mean()
print(f"Average Order Value: {avg_order_value:,.2f}")

# Repeat Purchase Rate
# Count orders per customer
order_counts = df["Order ID"].value_counts()
repeat_purchase_rate = (order_counts[order_counts > 1].count() / order_counts.count()) * 100
print(f"Repeat Purchase Rate: {repeat_purchase_rate:.2f}%")


   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in          Expedited  JNE3671  JNE3671-TU-XXXL   

        Category  ... currency Amount    ship-city   ship-state  \
0            Set  ...      INR 647.62

In [ ]:
E-commerce Sales Data Analysis

Analyzed 128k+ Amazon sales records using Python (Pandas, NumPy) and SQL to extract insights on top-selling products, monthly revenue trends, and top customers.

Calculated KPIs like Total Revenue, Average Order Value, and Repeat Purchase Rate to evaluate business performance.